<font color='green'> 
**Udemy - (50 Saat) Python A-Z™: Veri Bilimi ve Machine Learning**
11. Bölüm - Makine Öğrenmesi III - Doğrusal Olmayan Regresyon Modelleri (291-294. videolar) - Vahit Keskin anlattı. </font>

# K-nearest Neighbours Regression 

### İçindekiler

**Data Loading**

**Data Preprocessing**
* NaN değer bulunan satırları siliyoruz.
* y değerini belirliyoruz.
* 'League', 'Division', 'NewLeague' sütunları için one hot encoding yapıyoruz.
    1. Sadece ilgili sütunların tüm kategorilerinin yer aldığı bir dataseti oluşturuyoruz.
    2. One hot encoding yaptığımız orijinal sütunları ve y değişkeni olarak belirlediğimiz sütunu silip X olarak atıyoruz.
    3. `pd.concat()` fonksiyonuyla her sütunda bulunan kategori sayısısının 1 eksiği kadar (?) sütunu X'e dahil ediyoruz. 
    
* Elimizdeki son X ve y üzerinden train ve test setlerini ayırma işlemi gerçekleştiriyoruz.

**Creating Model with Scikit-Learn**

**Tahmin**
* Farklı k değerlerine karşılık olarak bu tahmin değerinin nasıl değiştiğini bir gözlemliyoruz.

**Model Tuning with GridSearch**
* GridSearch Nedir?
* GridSearch ile model kuruyoruz. 
* Neden GridSearch ile model kurma ihtiyacı duyduk?
* GridSearch ile yaptığımız model tuningde best parametreyi bulduktan sonra final modeli kuruyoruz. 
* Final model üzerinden test hatasını hesaplıyoruz.

## Data Loading

In [49]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [50]:
hit = pd.read_csv("Hitters.csv")
df = hit.copy()

In [51]:
df.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,NaN,A
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N


## Data Preprocessing

#### <font color='green'>NaN değer bulunan satırları siliyoruz.</font>

In [52]:
df = df.dropna()

#### <font color='green'>y değerini belirliyoruz.</font>

In [53]:
y = df["Salary"]

#### <font color='green'>'League', 'Division', 'NewLeague' sütunları için one hot encoding yapıyoruz.</font>

<font color='blue'> 1. Sadece ilgili sütunların tüm kategorilerinin yer aldığı bir dataseti oluşturuyoruz. </font>

In [54]:
dms = pd.get_dummies(df[['League','Division','NewLeague']])

In [55]:
dms.head()

,League_A,League_N,Division_E,Division_W,NewLeague_A,NewLeague_N
1,0,1,0,1,0,1
2,1,0,0,1,1,0
3,0,1,1,0,0,1
4,0,1,1,0,0,1
5,1,0,0,1,1,0


<font color='blue'> 2. One hot encoding yaptığımız orijinal sütunları ve y değişkeni olarak belirlediğimiz sütunu silip X olarak atıyoruz.</font>

In [56]:
X_ = df.drop(['Salary','League','Division','NewLeague'], axis=1).astype('float64')

In [57]:
X_.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors
1,315.0,81.0,7.0,24.0,38.0,39.0,14.0,3449.0,835.0,69.0,321.0,414.0,375.0,632.0,43.0,10.0
2,479.0,130.0,18.0,66.0,72.0,76.0,3.0,1624.0,457.0,63.0,224.0,266.0,263.0,880.0,82.0,14.0
3,496.0,141.0,20.0,65.0,78.0,37.0,11.0,5628.0,1575.0,225.0,828.0,838.0,354.0,200.0,11.0,3.0
4,321.0,87.0,10.0,39.0,42.0,30.0,2.0,396.0,101.0,12.0,48.0,46.0,33.0,805.0,40.0,4.0
5,594.0,169.0,4.0,74.0,51.0,35.0,11.0,4408.0,1133.0,19.0,501.0,336.0,194.0,282.0,421.0,25.0


<font color='blue'> 3. `pd.concat()` fonksiyonuyla her sütunda bulunan kategori sayısısının 1 eksiği kadar (?) sütunu X'e dahil ediyoruz.  </font>

In [58]:
X = pd.concat([X_, dms[['League_N','Division_W','NewLeague_N']]], axis=1)

In [59]:
X.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,League_N,Division_W,NewLeague_N
1,315.0,81.0,7.0,24.0,38.0,39.0,14.0,3449.0,835.0,69.0,321.0,414.0,375.0,632.0,43.0,10.0,1,1,1
2,479.0,130.0,18.0,66.0,72.0,76.0,3.0,1624.0,457.0,63.0,224.0,266.0,263.0,880.0,82.0,14.0,0,1,0
3,496.0,141.0,20.0,65.0,78.0,37.0,11.0,5628.0,1575.0,225.0,828.0,838.0,354.0,200.0,11.0,3.0,1,0,1
4,321.0,87.0,10.0,39.0,42.0,30.0,2.0,396.0,101.0,12.0,48.0,46.0,33.0,805.0,40.0,4.0,1,0,1
5,594.0,169.0,4.0,74.0,51.0,35.0,11.0,4408.0,1133.0,19.0,501.0,336.0,194.0,282.0,421.0,25.0,0,1,0


#### <font color='green'>Elimizdeki son X ve y üzerinden train ve test setlerini ayırma işlemi gerçekleştiriyoruz.</font>

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25, 
                                                    random_state=42)

In [61]:
X_train.shape

(197, 19)

In [62]:
X_test.shape

(66, 19)

In [63]:
y_train.shape

(197,)

In [64]:
y_test.shape

(66,)

## Creating Model with Scikit-Learn

In [65]:
knn_model = KNeighborsRegressor().fit(X_train, y_train)

In [66]:
knn_model

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                    weights='uniform')

<font color='blue'>Ön tanımlı olarak komşuluk parametresi değerini 5 olarak almış. Burada model hiperparametresi olarak adlandırabileceğimiz KNN'in parametresi komşuluk sayısıdır (n_neighbors). </font>

In [67]:
knn_model.n_neighbors

5

<font color='blue'>KNN'in farklı k komşuluk sayılarına karşılık farklı değerler vereceğini biliyoruz. Aynı zamanda bunun optimize etmemiz gereken parametre olduğunu da biliyoruz. Bunu optimize etme işlemini gerçekleştireceğiz.   </font>

## Tahmin

In [68]:
knn_model.predict(X_train) 
# eğitim setine yönelik olarak y tahmin değerlerini üretecek.
# X_train'deki her gözlem için yakınındaki 5 gözlemin y değerinin ortalamasını y olarak kabul etmiş olduk.

array([ 102.5   ,  952.    ,  904.    ,  325.5   ,  768.6666, 1180.6666,
        118.    ,   90.5   ,  708.6666,  840.4   , 1612.1142,  160.    ,
        982.    ,  771.    ,  716.6666,  745.    ,  670.    ,   79.5   ,
        247.5   ,  303.    ,  576.    ,  996.6666,  551.    ,  451.5   ,
         81.6   ,  962.5   ,  491.    ,  425.    ,   88.6   ,  463.    ,
         70.6   ,  680.3334, 1152.3332,   78.6   ,  433.3334,  103.5   ,
        281.2   ,  799.4   ,  106.5   ,   79.5   ,   79.5   ,  553.5   ,
        198.8334,  249.    ,  336.    , 1052.    ,  205.5   ,  126.3   ,
        637.    ,  602.    ,  498.    ,  248.    ,  322.    ,  160.    ,
       1103.6666,  148.4   ,  627.    ,  707.5   ,  220.    , 1472.1142,
        803.    ,  274.2   ,  588.    ,  487.5   ,  150.5   ,  158.3   ,
         84.    ,  694.5   ,  159.5   ,  303.5   ,  336.    ,  986.6666,
        737.5   ,  904.    , 1138.6666,  778.    ,  181.5   ,  498.    ,
        417.5   ,  840.4   ,  412.5   ,  510.    , 

In [69]:
knn_model.predict(X_test) 
# test setine yönelik olarak y tahmin değerlerini üretecek.

array([ 510.3334,  808.3334,  772.5   ,  125.5   , 1005.    ,  325.5   ,
        216.5   ,  101.5   ,  982.    ,  886.6666,  590.    ,  901.6666,
        831.6666,  157.5   ,  393.    , 1005.    ,  735.5   ,   97.    ,
        884.4   ,  302.    ,  450.    ,  817.6666,  832.6666,  392.3334,
        528.    ,   81.6   ,  735.    ,  470.    ,  722.5   ,  101.    ,
         90.5   ,   74.6   ,  748.3334,  217.    ,  280.5334, 1044.5   ,
        955.    ,  232.    ,   78.6   ,  529.    ,   77.6   ,  106.5   ,
        516.6666,  593.6666, 1005.    ,  649.1666,  715.    ,  101.5   ,
        134.5   ,  810.    ,  743.    ,  521.3334,  664.3334,  195.    ,
        102.4   ,  728.5   ,  488.    ,  962.5   ,  230.8334, 1040.    ,
        885.    ,  542.    ,  720.4   ,  571.    ,  735.    ,   81.6   ])

<font color='blue'>Test seti için tahmin ettiğimiz değerleri y_pred olarak kaydedip test hatasını hesaplayabiliriz. </font>

In [70]:
y_pred = knn_model.predict(X_test) 

In [71]:
np.sqrt(mean_squared_error(y_test, y_pred))

426.6570764525201

<font color='blue'>Test hatasını 426 olarak vermiş oldu. </font>

#### <font color='green'>Farklı k değerlerine karşılık olarak bu tahmin değerinin nasıl değiştiğini bir gözlemliyoruz.</font>

In [72]:
RMSE = [] 
for k in range(10):
    k = k+1
    knn_model = KNeighborsRegressor(n_neighbors = k).fit(X_train, y_train)
    y_pred = knn_model.predict(X_train) 
    rmse = np.sqrt(mean_squared_error(y_train,y_pred)) 
    RMSE.append(rmse)
    print("k =", k, "için RMSE değeri: ", rmse)

k = 1 için RMSE değeri:  0.0
k = 2 için RMSE değeri:  179.52761335480352
k = 3 için RMSE değeri:  205.20157172291863
k = 4 için RMSE değeri:  220.5139794876305
k = 5 için RMSE değeri:  239.64671325413764
k = 6 için RMSE değeri:  243.5904190007242
k = 7 için RMSE değeri:  258.1478781634636
k = 8 için RMSE değeri:  266.05374203349805
k = 9 için RMSE değeri:  269.73782093553376
k = 10 için RMSE değeri:  271.2798300436963


## Model Tuning with GridSearch

<font color='blue'>Bu bölümde grid search cv metoduyla optimum k sayısını belirleme işlemini gerçekleştireceğiz.  </font>

#### <font color='green'>GridSearch Nedir? </font>

<font color='blue'>Grid'in kelime anlamı ızgara. Öncelikle bütün makine öğrenmesi bölümlerinde bundan sonraki ve öncekiler için de GridSearch kullanılabilirdi fakat ElasticNet, Ridge Regression, Lasso Regression zaten özelleştirilmiş fonksiyonlar olduğundan dolayı GridSearch'u kullanmadık. Bundan sonraki bölümlerin hemen hepsinde GridSearch yöntemiyle hiperparametre araması yapacağız. GridSearch bir ızgara mantığıyla bir olası parametre setinin verilip tüm olası kombinasyonların denenmesi anlamına gelir. Örneğin 2 parametre olduğunda 2 parametre için bazı değerler aranmak istendiğinde bu 2 parametrenin birbirleriyle çarpıştırılarak olası tüm kombinasyonların denenmesi işlemini cross validation yöntemiyle gerçekleştirmek için kullanılan bir metoddur. Burada tek bir parametre ile biz bir grid oluşturacağız, bir liste oluşturacağız. Bu liste içerisinde bir arama işlemi gerçekleştirilecek.  </font>

### <font color='purple'>GridSearch ile model kuruyoruz. </font>

In [73]:
from sklearn.model_selection import GridSearchCV

In [74]:
np.arange(1,30,1) # 1'den 30'a kadar k değerleri üretmiş olduk. 

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])

In [75]:
knn_params = {'n_neighbors':np.arange(1,30,1)}
# 1'den 30'a kadar olan komşuluk değerlerini modelde deneyip 
# hangisinin daha iyi olduğu sonucuna gitmeye çalışacağız.

# Burada algoritmanın fonksiyonunun kabul ettiği parametrenin ismi (n_neighbors) 
# her nasıl ise o şekilde bir isimlendirme yapmak gerekiyor.

knn = KNeighborsRegressor() 
knn_cv_model = GridSearchCV(knn, knn_params, cv=10) # cv=10: 10 katlı cross validation yapıyoruz
knn_cv_model.fit(X_train,y_train)

GridSearchCV(cv=10, error_score=nan,
             estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30,
                                           metric='minkowski',
                                           metric_params=None, n_jobs=None,
                                           n_neighbors=5, p=2,
                                           weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

<font color='blue'>Model kurma işlemi gerçekleşti. Bu işlem cross validation yöntemine göre bize uygun olabilecek parametrelerin hepsini denedi ve bunun neticesinde uygun olan parametreyi bulmuş oldu. Bu en uygun parametreyi bulmak için `knn_cv_model.best_params_["n_neighbors"]` kodunu çalıştırıyoruz.</font>

In [76]:
knn_cv_model.best_params_["n_neighbors"]

8

#### <font color='green'>Neden GridSearch ile model kurma ihtiyacı duyduk?</font>

<font color='black'> *GridSearch ile yapmak Cross Validation ile yaptığımız anlamına geliyor. Burada kendimiz de GridSearchCV ile model kurmadan direkt kendi kurduğumuz model üzerinden cross validation skorlarını alabilirdik (R^2, RMSE vs.) çünkü tek parametre var sadece.*</font>

<font color='blue'>Cross validation yapmadan k değerlerine göre bulduğumuz RMSE değerlerini ve Cross validation yapılarak bulduğumuz RMSE değerlerini karşılaştırıyoruz. </font>

<font color='blue'>Tahmin bölümünde 10 k değeri için train seti üzerinden bir RMSE skoru hesaplamıştık, sadece train ve test olarak ayırıp değerleri almış olduk, valide etmemiştik. </font>

In [77]:
RMSE = [] 
for k in range(10):
    k = k+1
    knn_model = KNeighborsRegressor(n_neighbors = k).fit(X_train, y_train)
    y_pred = knn_model.predict(X_train) 
    rmse = np.sqrt(mean_squared_error(y_train,y_pred)) 
    RMSE.append(rmse)
    print("k =", k, "için RMSE değeri: ", rmse)

k = 1 için RMSE değeri:  0.0
k = 2 için RMSE değeri:  179.52761335480352
k = 3 için RMSE değeri:  205.20157172291863
k = 4 için RMSE değeri:  220.5139794876305
k = 5 için RMSE değeri:  239.64671325413764
k = 6 için RMSE değeri:  243.5904190007242
k = 7 için RMSE değeri:  258.1478781634636
k = 8 için RMSE değeri:  266.05374203349805
k = 9 için RMSE değeri:  269.73782093553376
k = 10 için RMSE değeri:  271.2798300436963


<font color='blue'>Şimdi valide edilmiş ve edilmemiş halleriyle iki tane RMSE değeri oluşturup bunların ikisi arasındaki bir durumu gözlemleyeceğiz. </font>

In [78]:
RMSE = [] 
RMSE_CV = []
for k in range(10):
    k = k+1
    knn_model = KNeighborsRegressor(n_neighbors = k).fit(X_train, y_train)
    y_pred = knn_model.predict(X_train) 
    rmse = np.sqrt(mean_squared_error(y_train,y_pred))
    RMSE.append(rmse)
    
    rmse_cv = np.sqrt(-1*cross_val_score(knn_model, X_train, y_train, cv=10, 
                                         scoring = "neg_mean_squared_error").mean())
    RMSE_CV.append(rmse_cv)
    
    print("k =" , k , "için RMSE değeri: ", rmse, "   RMSE_CV değeri: ", rmse_cv )

k = 1 için RMSE değeri:  0.0    RMSE_CV değeri:  325.39475147063825
k = 2 için RMSE değeri:  179.52761335480352    RMSE_CV değeri:  293.24000183333817
k = 3 için RMSE değeri:  205.20157172291863    RMSE_CV değeri:  283.7486667487823
k = 4 için RMSE değeri:  220.5139794876305    RMSE_CV değeri:  286.3240222024089
k = 5 için RMSE değeri:  239.64671325413764    RMSE_CV değeri:  290.0705466132226
k = 6 için RMSE değeri:  243.5904190007242    RMSE_CV değeri:  298.1263115575851
k = 7 için RMSE değeri:  258.1478781634636    RMSE_CV değeri:  294.77070479194987
k = 8 için RMSE değeri:  266.05374203349805    RMSE_CV değeri:  291.98672028891235
k = 9 için RMSE değeri:  269.73782093553376    RMSE_CV değeri:  295.7162739573105
k = 10 için RMSE değeri:  271.2798300436963    RMSE_CV değeri:  301.31047022701154


<font color='blue'>


- İlk RMSE değerleri train ve test üzerinden direkt yapılan hesaplamalar sonucu elde edilen hatalar.

- İkinci tarafta gördüğümüz RMSE_CV değerleri ise valide edilmiş model üzerinden elde edilen hatalar.

Ve bunların hepsi train hataları. 

- RMSE_CV değerlerine baktığımızda 290 ile 298 arasında değişmiş doğrulanmış bunlar. En küçük 283 iken en yüksek değer 301 aralarındaki fark 18.

- RMSE değerlerine baktığımızda ise 179'dan başlamış 243 var. Bu kadar fazla standart sapmalı bir tahmin değeri değişmesi çok makul değil. En küçük ve en yüksek değerleri arasında ise uçurum var. 


Burada küçük olanını aramıyoruz. Küçük olanı hep aldatıcı, yanıltıcı. 
k=2 için baktığımızda; RMSE değeri 179, RMSE_CV değeri ise 293 ama RMSE_CV değeri daha doğru. Çünkü modellemede kullanılan buradaki (ilk yaptığımız) setin 20'ye 80'lik herhangi bir seti. GridSearchCV, cross validation yönteminde ise data setinin her seferinde farklı bir yerini alıyor ve bu işlem sonucunda k'yı 8 bulmuşuz. 
    
RMSE_CV valide edildiği/doğrulandığı için modelinizin hatasını (eğitim üzerinden alacak olduğunuz hata değerlendirmesini ve nihayetinde kuracak olup değerlendirmek için kullanacağınız test hatasını) mutlaka valide edilmiş model üzerinden değerlendirin. 

    
</font>

### <font color='purple'>GridSearch ile yaptığımız model tuningde best parametreyi bulduktan sonra final modeli kuruyoruz. </font>

#### <font color='green'>Modeli kuruyoruz ve eğitiyoruz.</font>

In [79]:
knn_tuned = KNeighborsRegressor(n_neighbors = knn_cv_model.best_params_["n_neighbors"])
knn_tuned.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=8, p=2,
                    weights='uniform')

#### <font color='green'>Bu model üzerinden test hatasını hesaplıyoruz.</font>

<font color='blue'>`knn_tuned.predict(X_test)` ile tahmin edilen y değerlerini buluyoruz sonrasında hata skorunu hesaplıyoruz.</font>

In [80]:
np.sqrt(mean_squared_error(y_test, knn_tuned.predict(X_test)))

413.7094731463598